In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from prettytable import PrettyTable
from IPython.display import display

In [ ]:
tickers = ["BHP.AX", "GOOGL", "AMZN", "WMT", "XOM", "BRK-A", "UNH", "UPS", "AMT", "AAPL", "NEE"]

# Download historical data from Yahoo Finance
data = yf.download(tickers, start="2019-02-01", end="2021-12-30")

# Filter the "Adj Close" column for daily returns
returns = data["Adj Close"].pct_change()

# Define a dictionary with ticker and company names
companies = {
    "BHP.AX": "BHP Group Limited",
    "GOOGL": "Alphabet.com, Inc.",
    "AMZN": "Amazon.com, Inc.",
    "WMT": "Walmart Inc.",
    "XOM": "Exxon Mobil Corporation",
    "BRK-A": "Berkshire Hathaway Inc.",
    "UNH": "UnitedHealth Group Incorporated",
    "UPS": "United Parcel Service, Inc.",
    "AMT": "American Tower Corporation",
    "AAPL": "Apple Inc.",
    "NEE": "NextEra Energy, Inc."
}